# Homework 3: Language Modelling in Hangman

Student Name:Junxu Zhang

Student ID:901264

## General info

<b>Due date</b>:  Friday, 17 May 2019 4pm

<b>Submission method</b>: see LMS

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -20% per day

<b>Marks</b>: 7% of mark for class (with 6% on correctness + 1% on quality and efficiency of your code)

<b>Materials</b>: See the main class LMS page for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib and Scikit-Learn. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages; if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3</b>. 

To familiarize yourself with NLTK, here is a free online book:  Steven Bird, Ewan Klein, and Edward Loper (2009). <a href=http://nltk.org/book>Natural Language Processing with Python</a>. O'Reilly Media Inc. You may also consult the <a href=https://www.nltk.org/api/nltk.html>NLTK API</a>.

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Updates</b>: Any major changes to the assignment will be announced via LMS. Minor changes and clarifications will be announced in the forum on LMS, we recommend you check the forum regularly.

<b>Academic Misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.


## Overview

In this homework, you'll be creating an 'artificial intelligence' player for the classic Hangman word guessing game. You will need to implement several different automatic strategies based on character level language models. Your objective is to create an automatic player which makes the fewest mistakes.

## The Hangman Game (*No implementation is needed*)

The <a href="https://en.wikipedia.org/wiki/Hangman_(game)">Hangman game</a> is a simple game whereby one person thinks of a word, which they keep secret from their opponent, who tries to guess the word one character at a time. The game ends when the opponent makes more than a fixed number of incorrect guesses, or they figure out the secret word before then (in which case they *win*). 

Here's a simple version of the game. 

In [2]:
def hangman(secret_word, guesser, max_mistakes=8, verbose=True, **guesser_args):
    """
        This function plays the hangman game with the provided gusser and returns the number of incorrect guesses. 
        
        secret_word: a string of lower-case alphabetic characters, i.e., the answer to the game
        guesser: a function which guesses the next character at each stage in the game
            The function takes a:
                mask: what is known of the word, as a string with _ denoting an unknown character
                guessed: the set of characters which already been guessed in the game
                guesser_args: additional (optional) keyword arguments, i.e., name=value
        max_mistakes: limit on length of game, in terms of allowed mistakes
        verbose: be chatty vs silent
        guesser_args: keyword arguments to pass directly to the guesser function
    """
    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word)
    guessed = set()
    if verbose:
        print("Starting hangman game. Target is", ' '.join(mask), 'length', len(secret_word))
    
    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print("You have", (max_mistakes-mistakes), "attempts remaining.")
        guess = guesser(mask, guessed, **guesser_args)

        if verbose:
            print('Guess is', guess)
        if guess in guessed:
            if verbose:
                print('Already guessed this before.')
            mistakes += 1
        else:
            guessed.add(guess)
            if guess in secret_word:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                if verbose:
                    print('Good guess:', ' '.join(mask))
            else:
                if verbose:
                    print('Sorry, try again.')
                mistakes += 1
                
        if '_' not in mask:
            if verbose:
                print('Congratulations, you won.')
            return mistakes
        
    if verbose:
        print('Out of guesses. The word was', secret_word)    
    return mistakes

Here is a human guesser allowing interactive play.

In [3]:
def human(mask, guessed, **kwargs):
    """
    This is a simple function for manual play.
    """
    print('Enter your guess:')
    return 

If you want to play hangman interactively, please set `interactive` to True. When submitting your solution, set to False so we can automatically run the whole notebook using `Run All`.

In [4]:
interactive = True

<b>For your testing:</b>

You can play the game interactively using the following command:

In [5]:
if interactive:
    hangman('whatever', human, 8, True)

Starting hangman game. Target is _ _ _ _ _ _ _ _ length 8
You have 8 attempts remaining.
Enter your guess:
Guess is None


TypeError: 'in <string>' requires string as left operand, not NoneType

## 1. Preparing Test Set and Training Set (1 mark)

<b>Instructions</b>: We will be using the words occurring in the *Brown* corpus for *training* an artificial intelligence guessing algorithm, and for *evaluating* the quality of the algorithm. Note that we are intentionally making the hangman game hard, as the AI will need to cope with test words that it has not seen before, hence it will need to learn generalisable patterns of characters to make reasonable predictions.

Your first task is to compute the unique word types occurring in the *Brown* corpus, using `nltk.corpus.Brown` and the `words` method, selecting only words that are entirely comprised of alphabetic characters, and lowercasing the words. Finally, randomly shuffle (`numpy.random.shuffle`) this collection of word types, and split them into disjoint training and testing sets. The test set should contain 1000 word types, and the rest should be in the training set. Your code should print the sizes of the training and test sets.

Feel free to test your own Hangman performance using `hangman(numpy.random.choice(test_set), human, 8, True)`. It is surprisingly difficult (and addictive)!

(1 mark)

In [36]:
from nltk.corpus import brown
import numpy as np

np.random.seed(12345)

# word_set stores all the unique word types in the Brown corpus
word_set = []
# test_set stores 1000 word types for testing
test_set = []
# training_set stores the rest word types for training
training_set = []

###
# Your answer BEGINS HERE
###
#print(brown.paras()[:2])
#print(brown.words())
for word in brown.words():
    if word.isalpha():     
        word = word.lower()
        if word not in word_set:
            word_set.append(word)
       # print(sentences,1)
      #  for words in sentences:
      #      pass
#print(word_set)
np.random.shuffle(word_set)

test_set = word_set[-1000:]
training_set = word_set[:-1000]

###
# Your answer ENDS HERE
###

print(len(word_set))
print(len(test_set))
print(len(training_set))

40234
1000
39234


<b>For your testing:</b>

In [37]:
assert(len(word_set) > 35000 and len(word_set) < 45000)

In [38]:
assert(len(test_set) == 1000)

In [39]:
assert(len(training_set) + len(test_set) == len(word_set))

In [40]:
if interactive:
    hangman(np.random.choice(test_set), human, 8, True)

Starting hangman game. Target is _ _ _ _ _ _ _ _ _ length 9
You have 8 attempts remaining.
Enter your guess:
Guess is None


TypeError: 'in <string>' requires string as left operand, not NoneType

## 2. Simple Guesser: Random Guessing (1 mark)

<b>Instructions</b>: To set a baseline, your first *AI* attempt will be a trivial random method. For this you should implement a guessing method, similar to the `human` method above, i.e., using the same input arguments and returning a character. Your method should randomly choose a character from the range `'a'...'z'` after excluding the characters that have already been guessed in the current game (all subsequent AI approaches should also exclude previous guesses). You might want to use `numpy.random.choice` for this purpose.

To help you measure the performance of this (and later) guesser, a `test_guesser` method that takes a guesser and measures the average number of incorrect guesses made over all the words in the `test_set` is provided to you. 

(1 mark)

In [41]:
def test_guesser(guesser, test=test_set):
    """
        This function takes a guesser and measures the average number of incorrect guesses made over all the words in the test_set. 
    """
    total = 0
    for word in test:
        total += hangman(word, guesser, 26, False)
    return total / float(len(test))

In [42]:
import string

def random_guesser(mask, guessed, **kwargs):
    """
        This function implements a random guesser. It returns the random guess. 
    """
    ###
    # Your answer BEGINS HERE
    ###
    scope = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
    temp = np.random.choice(scope)
    while temp in guessed:
        temp = np.random.choice(scope)
    #print(guessed)
    return temp
        
    ###
    # Your answer ENDS HERE
    ###

# uncomment to run a single hangman game with output shown (useful for debugging)
#hangman(np.random.choice(test_set), random_guesser, 10, True)

result = test_guesser(random_guesser)
print()
print("Average number of incorrect guesses: ", result)


Average number of incorrect guesses:  16.671


<b>For your testing:</b>

In [43]:
assert(result > 10 and result < 20)

## 3. Your First AI Guesser: Unigram Guesser (1 mark)

**Instructions:** As your first real AI, you should train a *unigram* model over the training set.  This requires you to find the frequencies of characters over all training words. Using this model, you should write a guesser that returns the character with the highest probability. Remember to exclude already guessed characters. 

Hint: It should be much lower than random guessing.

(1 mark)

In [44]:
from collections import Counter

# unigram_counts stores the frequencies of characters over all training words
unigram_counts = Counter()

###
# Your answer BEGINS HERE
###
total = ""
for term in training_set:
    total += term
unigram_counts['a'] = total.count('a')
unigram_counts['b'] = total.count('b')
unigram_counts['c'] = total.count('c')
unigram_counts['d'] = total.count('d')
unigram_counts['e'] = total.count('e')
unigram_counts['f'] = total.count('f')
unigram_counts['g'] = total.count('g')
unigram_counts['h'] = total.count('h')
unigram_counts['i'] = total.count('i')
unigram_counts['j'] = total.count('j')
unigram_counts['k'] = total.count('k')
unigram_counts['l'] = total.count('l')
unigram_counts['m'] = total.count('m')
unigram_counts['n'] = total.count('n')
unigram_counts['o'] = total.count('o')
unigram_counts['p'] = total.count('p')
unigram_counts['q'] = total.count('q')
unigram_counts['r'] = total.count('r')
unigram_counts['s'] = total.count('s')
unigram_counts['t'] = total.count('t')
unigram_counts['u'] = total.count('u')
unigram_counts['v'] = total.count('v')
unigram_counts['w'] = total.count('w')
unigram_counts['x'] = total.count('x')
unigram_counts['y'] = total.count('y')
unigram_counts['z'] = total.count('z')
#unigram_counts= dict(Counter(total))
#unigram_counts = sorted(unigram_counts.values())
#print(unigram_counts.most_common(1)[0][0])
#print(len(unigram_counts))
###
# Your answer ENDS HERE
###
print(unigram_counts)

def unigram_guesser(mask, guessed, unigram_counts=unigram_counts):
    """
        This function implements a unigram guesser. It returns a guess based on the unigram model. 
    """
    ###
    # Your answer BEGINS HERE
    ###
    temp = sorted(unigram_counts, key = unigram_counts.get, reverse = True)
    #print(temp,len(temp))
    result = list(filter(lambda x : x not in guessed, temp))
    #print(mask)
    return result[0] 
    ###
    # Your answer ENDS HERE
    ###

#hangman(np.random.choice(test_set), unigram_guesser, 10, True)

result = test_guesser(unigram_guesser)
print()
print("Average number of incorrect guesses: ", result)

Counter({'e': 35230, 'i': 26056, 'a': 24346, 's': 23722, 'n': 22615, 'r': 22078, 't': 20612, 'o': 19164, 'l': 16674, 'c': 12606, 'd': 12247, 'u': 10192, 'm': 8696, 'g': 8612, 'p': 8537, 'h': 7358, 'b': 5732, 'y': 5332, 'f': 4228, 'v': 3427, 'k': 2991, 'w': 2791, 'z': 1036, 'x': 838, 'j': 646, 'q': 529})

Average number of incorrect guesses:  10.156


<b>For your testing:</b>

In [45]:
assert(result > 5 and result < 15)

## 4. Your Second AI Guesser: Length-based Unigram Guesser (1 mark)

**Instructions:** The length of the secret word is an important clue that we might exploit. Different length words tend to have different distributions over characters, e.g., short words are less likely to have suffixes or prefixes. You should incorporate this idea by conditioning the unigram model on the length of the secret word, i.e., having a *different* unigram model for each length of the words. You will need to be a little careful at test time, to be robust to the (unlikely) situation that you encounter a word length that you didn't see in training. You need to decide on how to handle this situation.

(1 mark)

In [46]:
from collections import defaultdict

# unigram_counts_by_length stores a dictionary, mapping word length to the frequencies of characters of words with that word length
unigram_counts_by_length = defaultdict(Counter)

###
# Your answer BEGINS HERE
###

for word in training_set:
    if len(word) not in unigram_counts_by_length.keys():
        unigram_counts_by_length[len(word)] = Counter(word)
    if len(word) in unigram_counts_by_length.keys():
        unigram_counts_by_length[len(word)] += Counter(word)

###
# Your answer ENDS HERE
###

lengths = sorted(unigram_counts_by_length.keys())
max_length = lengths[-1] + 1
print(lengths)
print()
print(unigram_counts_by_length)

def unigram_length_guesser(mask, guessed, counts=unigram_counts_by_length):
    """
        This function implements a length-based unigram guesser. It returns a guess based on the length-based unigram model. 
    """
    ###
    # Your answer BEGINS HERE
    ###
    #temp = sorted(unigram_counts, key = unigram_counts.get, reverse = True)

    scope = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
    for k in dict(counts):
            #print(k)
            if len(mask) == k:
                temp = sorted(dict(counts)[k], key = dict(counts)[k].get, reverse = True)
                
                break
       
    result = list(filter(lambda x : x not in guessed, temp))
    if result != []:
        return result[0]
    else:
        return list(filter(lambda x : x not in guessed, scope))[0] 
    ###
    # Your answer ENDS HERE
    ###

#hangman(np.random.choice(test_set), unigram_length_guesser, 10, True)

result = test_guesser(unigram_length_guesser)
print()
print("Average number of incorrect guesses: ", result)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]

defaultdict(<class 'collections.Counter'>, {6: Counter({'e': 4246, 'a': 2685, 's': 2591, 'r': 2533, 'i': 2032, 'n': 1996, 'o': 1983, 'l': 1958, 't': 1846, 'd': 1619, 'u': 1144, 'c': 1065, 'm': 938, 'g': 882, 'h': 878, 'p': 853, 'b': 772, 'y': 707, 'f': 503, 'k': 456, 'w': 432, 'v': 395, 'z': 123, 'j': 106, 'x': 93, 'q': 50}), 11: Counter({'e': 2965, 'i': 2773, 'n': 2280, 't': 2164, 'a': 2029, 's': 2005, 'r': 1922, 'o': 1696, 'l': 1323, 'c': 1239, 'u': 854, 'd': 801, 'p': 776, 'm': 743, 'g': 642, 'h': 517, 'b': 420, 'y': 410, 'f': 293, 'v': 293, 'k': 134, 'w': 117, 'x': 72, 'z': 70, 'q': 44, 'j': 27}), 7: Counter({'e': 5336, 'a': 3534, 's': 3484, 'r': 3419, 'i': 3329, 'n': 3034, 't': 2649, 'o': 2501, 'l': 2485, 'd': 2054, 'c': 1669, 'u': 1512, 'g': 1498, 'm': 1241, 'p': 1229, 'h': 1106, 'b': 946, 'y': 730, 'f': 678, 'k': 587, 'w': 519, 'v': 457, 'z': 157, 'x': 110, 'j': 98, 'q': 81}), 8: Counter({'e': 5589,

<b>For your testing:</b>

In [47]:
assert(result > 5 and result < 15)

## 5. Your Third AI Guesser: Bigram Guesser (1 mark)

**Instructions:** Now for the next challenge, using a *bigram* language model over characters. The order of characters is obviously important, yet this wasn't incorporated in any of the above models. Knowing that the word has the sequence `n _ s s` is a pretty strong clue that the missing character might be `e`. Similarly the distribution over characters that start or end a word are highly biased (e.g., toward common prefixes and suffixes, like *un-*, *-ed* and *-ly*).

You should develop a *bigram* language model over characters, train this over the training words (being careful to handle the start of each word properly, e.g., by padding with a sentinel symbol `$`.) You should use *linear interpolation* to smooth between the higher order and lower order models, and you will have to decide how to weight each component (be reminded that all probabilities should sum to 1).

Your bigram guesser should apply your language model to each blank position in the secret word by using the left context as is known. E.g., in the partial word `$ _ e c _ e _ _` we know the left context for the first three blanks, but have no known left context for the last blank. Using a bigram language model, we are able to apply it to the first three blanks only. You should then select the character with the highest probability of predicting the most number of correct entries over the blanks. 

Do you see any improvement over the unigram guessers above?

(1 mark)

In [48]:
###
# Your answer BEGINS HERE
###
#for bigram model the start tag and end tag is required. 
#start tag is represented by $, the end tag is represented by # ?No need for end tag, the end tag only used in generate sentence
import math
def convert_word(word):
    return ['$']+[c for c in word]

bigram_counts = defaultdict(Counter) # you will want a different data structure to store the bigram 

for train_word in training_set:
   # word_chars=list(train_word)
    word_chars=convert_word(train_word)
    bigram_list=zip(word_chars[:-1],word_chars[1:]) 
    for bigram in bigram_list:
        first,second=bigram
        bigram_counts[first][second] += 1

def get_prob_interp(prev_c,cur_c,unigram_counts=unigram_counts,bigram_counts=bigram_counts,lambdas=(0.79,0.21)):
    bigram_lambda=lambdas[0]
    unigram_lambda=lambdas[1] 
    sm_bigram_counts=bigram_counts[prev_c][cur_c]*bigram_lambda
    if sm_bigram_counts==0.0:
        interp_bigram_pp=0
    else:
        if prev_c=='$':
            u_counts=len(training_set)
        else:
            u_counts=unigram_counts[prev_c]
        interp_bigram_pp=sm_bigram_counts/float(u_counts)
    
    total_char_occurance=sum(unigram_counts.values())
    interp_unigram_pp=(unigram_counts[cur_c]/total_char_occurance)*unigram_lambda
    return math.log(interp_bigram_pp+interp_unigram_pp)
    
###
# Your answer ENDS HERE
###

def bigram_guesser(mask, guessed, counts=bigram_counts): 
    """
        This function implements a bigram guesser. It returns a guess based on the bigram model using linear interpolation.
    """
    ###
    # Your answer BEGINS HERE
    ###
  
    bigram_pp=Counter()
    temp_mask=convert_word(mask)
    
    candidate_chars=[]
    for char in list(string.ascii_lowercase):
        if char not in guessed:
            candidate_chars.append(char)
    
    for cur_c_index in range(1,len(temp_mask)):
        if temp_mask[cur_c_index]=='_' and temp_mask[cur_c_index-1]!='_': 
        #if temp_mask[cur_c_index]=='_' and temp_mask[cur_c_index-1]!='_': 
            for char in candidate_chars:
                bigram_pp[temp_mask[cur_c_index-1]+char]=get_prob_interp(temp_mask[cur_c_index-1],char)

    return_key = list(max(bigram_pp,key=bigram_pp.get))[1]
    #print(list(max(bigram_pp,key=bigram_pp.get)))
    return return_key

    ###
    # Your answer ENDS HERE
    ###
result = test_guesser(bigram_guesser)
#print(dic)
#print()
print("Average number of incorrect guesses: ", result)

Average number of incorrect guesses:  8.728


<b>For your testing:</b>

In [49]:
assert(result < 13)

## 6. Your Own AI Guesser (1 mark)

**Instructions:** You should try to develop a more effective AI, `my_ai_guesser`, for hangman. Feel free to engage your creativity here! Possibilities include better conditioning on the length of the word, fancier smoothing methods, and using ngram models. Ensure you report the test performance of your guesser. Have fun!

You will be marked based on the explanation of your approach and its accuracy. 

(1 mark) 

In [50]:
###
# Your answer BEGINS HERE
###
def convert_word_trigram(word):
    return ['$']+['$']+[c for c in word]

trigram_counts = defaultdict(Counter) 
#generate the bigram_counts
for train_word in training_set:
    word_chars=list(train_word)
    word_chars=convert_word_trigram(word_chars)
    trigram_list=zip(word_chars[:-2],word_chars[1:-1],word_chars[2:]) 
    for trigram in trigram_list:
        fc,sc,tc=trigram
        trigram_counts[fc+sc][tc]+=1
    

def get_trigram_prob_interp(prev_2c,cur_c,unigram_counts=unigram_counts,bigram_counts=bigram_counts,trigram_counts=trigram_counts,lambdas=(0.6,0.2,0.2)):
    trigram_lambda=lambdas[0]
    bigram_lambda=lambdas[1]
    unigram_lambda=lambdas[2] #the sum of them two lambdas should be 1
    
    #trigram model interpolation part probability
    if trigram_counts[prev_2c][cur_c]==0:
        interp_trigram_pp=0
    else:
        if prev_2c=='$$':
            prev_bi_counts=len(training_set)
        else:
            c1,c2=tuple(prev_2c)
            prev_bi_counts=bigram_counts[c1][c2]
        if prev_bi_counts!=0:
            interp_trigram_pp=trigram_counts[prev_2c][cur_c]/float(prev_bi_counts)*trigram_lambda
        else:
            interp_trigram_pp=0
    
    c1,c2=tuple(prev_2c)
    #get the bigram probability
    sm_bigram_counts=bigram_counts[c1][c2]*bigram_lambda
    if sm_bigram_counts==0.0:
        interp_bigram_pp=0
    else:
        if c1=='$':
            u_counts=len(training_set)
        else:
            u_counts=unigram_counts[c1]
        interp_bigram_pp=sm_bigram_counts/float(u_counts)
    
    total_char_occurance=sum(unigram_counts.values())
    
    interp_unigram_pp=(unigram_counts[cur_c]/total_char_occurance)*unigram_lambda
    
    return interp_trigram_pp+interp_bigram_pp+interp_unigram_pp
    

def my_ai_guesser(mask, guessed):
    trigram_pp=Counter()
    temp_mask=convert_word_trigram(mask)
    
    candidate_chars=[]
    for char in list(string.ascii_lowercase):
        if char not in guessed:
            candidate_chars.append(char)
            
    for cur_c_index in range(2,len(temp_mask)): 
        if temp_mask[cur_c_index]=='_' and temp_mask[cur_c_index-1]!='_'and temp_mask[cur_c_index-2]!='_': #means the bigram model could be applied.
            for char in candidate_chars:
                trigram_pp[temp_mask[cur_c_index-2]+temp_mask[cur_c_index-1]+char]=get_trigram_prob_interp(temp_mask[cur_c_index-2]+temp_mask[cur_c_index-1],char)
    
    return_key = list(max(trigram_pp,key=trigram_pp.get))[2]
    return return_key

###
# Your answer ENDS HERE
###

result = test_guesser(my_ai_guesser)
print()
print("Average number of incorrect guesses: ", result)


Average number of incorrect guesses:  8.247


**Instructions:** Explain your approach and discuss your result below. Please keep it brief.